## LLM Model

In [2]:
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.llms.ollama import Ollama

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

# llm = LlamaCPP(
#     # You can pass in the URL to a GGML model to download it automatically
#     model_url=model_url,
#     # optionally, you can set the path to a pre-downloaded model instead of model_url
#     model_path=None,
#     temperature=0.1,
#     max_new_tokens=256,
#     # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
#     context_window=3900,
#     # kwargs to pass to __call__()
#     generate_kwargs={},
#     # kwargs to pass to __init__()
#     # set to at least 1 to use GPU
#     model_kwargs={"n_gpu_layers": 5},
#     verbose=True,
# )

from llama_index.embeddings.ollama import OllamaEmbedding

embed_model = OllamaEmbedding(
    model_name="codellama",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)

llm = Ollama(
    model="codellama", 
    request_timeout=120.0,
    system_prompt="You are a senior developer specializing in static analysis tools and secure operating systems. You will follow the input instructions and generate code. Your output will be purely code, without additional information.",
    temperature=0.1
)

## Initialize Postgres

In [3]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "123456"
port = "5432"
user = "evan"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute("""
        SELECT pg_terminate_backend(pg_stat_activity.pid)
        FROM pg_stat_activity
        WHERE pg_stat_activity.datname = 'vector_db'
        AND pid <> pg_backend_pid();
    """)
    c.execute("DROP DATABASE IF EXISTS vector_db;")
conn.close()

conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute("CREATE DATABASE vector_db;")

In [4]:
from sqlalchemy import make_url
from llama_index.vector_stores.postgres import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="codellama",
    embed_dim=4096,  # llama embedding dimension
)

## Build an Ingestion Pipeline from Scratch

In [6]:
from llama_index.readers.file import PyMuPDFReader

In [21]:
loader = PyMuPDFReader()
documents = loader.load(file_path="./data/sootsurvivorsguide.pdf")

from llama_index.core import SimpleDirectoryReader
required_exts = [".md", ".java"]
reader = SimpleDirectoryReader(
    input_dir="/home/ubuntu/ConfigCraft.ai/experimental/data/soot_doc",
    required_exts=required_exts,
    recursive=True,
)
docs = reader.load_data()
print(f"Loaded {len(docs)} docs")

# combine the two document lists
documents = documents + docs

Loaded 29 docs


## Use a Text Splitter to Split Documents

In [23]:
from llama_index.core.node_parser import SentenceSplitter

In [24]:
text_parser = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [25]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_parser.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))


## Manually Construct Nodes from Text Chunks

In [26]:
from llama_index.core.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

In [32]:
print(f"Loaded {len(nodes)} nodes")

Loaded 87 nodes


## Generate Embeddings for each Node

In [33]:
import concurrent.futures

def get_and_set_embedding(node):
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding
    return node

with concurrent.futures.ThreadPoolExecutor() as executor:
    nodes = list(executor.map(get_and_set_embedding, nodes))

## Load Nodes into a Vector Store

In [34]:
vector_store.add(nodes)

['15fff635-bc96-4f7e-974d-54f5f1e6c09b',
 '99afa511-36da-46ef-abbf-381486449723',
 '5656bc28-f806-4ce4-a4bc-c45f41ce690e',
 'a937b3f0-e105-426a-9c16-49019cc03811',
 'a2ced6c5-0524-46d6-a414-ec6e1ba6188d',
 '48251e1c-9320-410b-87b5-ce2c58801b8d',
 '18bf7c64-17ad-4ffe-b03a-c83542fe5120',
 '0f42686b-f8d7-4e63-b40b-03f3d1c469e6',
 '309ef8d1-cf03-46a5-8093-7727437adf29',
 'e0a4a277-d445-4c9c-bfc4-200df30369e0',
 '630f8b94-7ccd-49f6-bd1a-4f7cf74cf40f',
 '372c2dd0-d9f3-4685-9203-4af701a18ffe',
 '0748889e-8f11-40e0-a313-caf045aba30a',
 '43f61347-08a4-4f72-bf80-eb6d1f404469',
 'f20f7694-a72c-47a1-8c90-3aba4982d427',
 '13f7cdb9-0d93-4745-befb-d23f692e14aa',
 '13109f01-e3bb-46ae-a725-1cfee9966326',
 '2f223b47-7fce-49a0-9477-e1a15a71ff3d',
 '1a0706d2-0761-4193-bc7b-6f5172b4b83b',
 '61de95ef-7582-4086-9555-59807d97e6d4',
 '0c0fbde5-b02c-459b-b984-a5ab34dbc5b2',
 '01fb6de0-1bd4-4f5e-90c2-be720257097e',
 '253829ed-79cf-41e1-bc68-858352e969c3',
 '186b8b06-062f-418e-bd3a-a477e84d7851',
 '313ab7b4-2660-

### Put into a Retriever

In [36]:
from llama_index.core import QueryBundle
from llama_index.core.retrievers import BaseRetriever
from typing import Any, List
from llama_index.core.schema import NodeWithScore
from typing import Optional
from llama_index.core.vector_stores import VectorStoreQuery


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k
        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(
            query_bundle.query_str
        )
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [37]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode

retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode=VectorStoreQueryMode.DEFAULT, similarity_top_k=2
)

## Plug this into our RetrieverQueryEngine to synthesize a response

In [38]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(retriever, llm=llm)

In [39]:
# query_str = "Create an iptables config for a web server 127.1.1.1 that only allows incoming traffic on port 80 and 443."
# query_str = "Generate Soot (verification) command that prints CFGs for all functions in java.util.concurrent.ThreadPoolExecutor class."
# query_str = "Generate an iptables config for an ecommerce website server that only allows incoming HTTP/HTTPS traffic, DNS lookup and SSH connecton."
query_str = "Generate Soot (verification) driver code that prints CFGs for all functions in `java.util.concurrent.ThreadPoolExecutor`"

response = query_engine.query(query_str)

In [40]:
print(str(response))

```
import java.util.concurrent.ThreadPoolExecutor;
import soot.*;
import soot.jimple.*;
import soot.toolkits.graph.*;

public class SootDriver {
    public static void main(String[] args) {
        // Initialize Soot
        G.v().out.println("Initializing Soot...");
        Scene scene = new Scene();
        scene.addBasicClass("java.util.concurrent.ThreadPoolExecutor", SootClass.BODIES);

        // Get the ThreadPoolExecutor class
        SootClass threadPoolExecutorClass = scene.getSootClass("java.util.concurrent.ThreadPoolExecutor");

        // Iterate over all methods in the class
        for (SootMethod method : threadPoolExecutorClass.getMethods()) {
            // Skip non-static methods
            if (!method.isStatic()) continue;

            // Get the body of the method
            SootBody body = method.retrieveActiveBody();

            // Create a new CFG for the method
            ControlFlowGraph cfg = new ControlFlowGraph(body);

            // Print the CFG to st

In [43]:
print(response.source_nodes[0].get_content())

With Docker

- Use the Docker image from docker hub (`docker run -it noidsirius/soot_tutorial:latest`) or
- Build the image locally (`docker build . -t soot_tutorial`) and run it (`docker run -it soot_tutorial`)
